# Unsloth Environment Verification

This notebook verifies that all components are correctly installed for running Unsloth notebooks:
- GRPO (Reinforcement Learning)
- Vision fine-tuning (Ministral VL)
- fast_inference support

**Run this after rebuilding the jupyter pod to verify the environment.**

**Note:** This notebook does NOT use vLLM fast_inference. For fast_inference tests, see:
- `01_FastInference_Llama.ipynb` - Llama-3.2-1B
- `02_FastInference_Qwen.ipynb` - Qwen3-4B
- `03_FastInference_Ministral.ipynb` - Ministral models
- `04_Vision_Training.ipynb` - Vision training pipeline

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load .env from notebook directory
load_dotenv()
print(f"✓ HF_TOKEN loaded: {'Yes' if os.environ.get('HF_TOKEN') else 'No'}")

# CRITICAL: Import unsloth FIRST for proper TRL patching
import unsloth
from unsloth import FastLanguageModel, FastVisionModel
print(f"✓ unsloth: {unsloth.__version__}")

import transformers
print(f"✓ transformers: {transformers.__version__}")

import vllm
print(f"✓ vLLM: {vllm.__version__}")

import trl
print(f"✓ TRL: {trl.__version__}")

import torch
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Test imports for Reinforcement Learning notebook
print("=== GRPO/RL Imports ===")
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
print("✓ All GRPO imports successful")

In [ ]:
# Test imports for Vision notebook
print("=== Vision/SFT Imports ===")
from trl import SFTTrainer, SFTConfig
from unsloth.trainer import UnslothVisionDataCollator
from unsloth import is_bf16_supported
from transformers import TextStreamer
print("✓ All Vision imports successful")

In [ ]:
# Test model loading with FastLanguageModel (standard inference, no vLLM)
print("=== Testing Model Loading ===")
model, tokenizer = None, None
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        "unsloth/Ministral-3-3B-Reasoning-2512-bnb-4bit",
        max_seq_length=2048,
        load_in_4bit=True,
    )
    print(f"✓ Model loaded: {type(model).__name__}")
    print(f"✓ Tokenizer: {type(tokenizer).__name__}")
    print("✓ FastLanguageModel test PASSED")
finally:
    del model, tokenizer

In [ ]:
# Test fast_inference capability
print("=== Fast Inference Check ===")
print("fast_inference=True uses vLLM as backend for 2x faster inference")
print(f"vLLM version: {vllm.__version__}")
print(f"Unsloth version: {unsloth.__version__}")

# Check if fast_inference is supported
import inspect
sig = inspect.signature(FastLanguageModel.from_pretrained)
if 'fast_inference' in sig.parameters:
    print("✓ fast_inference parameter available")
else:
    print("⚠ fast_inference parameter not found")

# Check FastVisionModel
sig_vision = inspect.signature(FastVisionModel.from_pretrained)
has_fast_inference_vision = 'fast_inference' in sig_vision.parameters
print(f"✓ fast_inference in FastVisionModel: {has_fast_inference_vision}")

In [ ]:
# Shutdown kernel to release all GPU memory
import IPython
print("Shutting down kernel to release GPU memory...")
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

## Verification Summary

If all cells above ran without errors, your environment is ready for:

1. **Reinforcement Learning** (GRPO/TRL imports verified)
2. **Vision Fine-tuning** (SFT imports verified)
3. **fast_inference** (parameter available)

### Next Steps

Run the specialized fast_inference notebooks (each requires fresh kernel):
- `01_FastInference_Llama.ipynb` - Llama-3.2-1B with vLLM
- `02_FastInference_Qwen.ipynb` - Qwen3-4B with vLLM
- `03_FastInference_Ministral.ipynb` - Ministral models
- `04_Vision_Training.ipynb` - Complete vision training pipeline